<a href="https://colab.research.google.com/github/jgonzet/Data-Management-7605-/blob/master/Spark%20Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalamos e importamos librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con la colección de Shakespeare

In [ ]:
downloaded = drive.CreateFile({'id':"1ybtSQxrqVqbRrl_3FMzMYW03Flp4zM-j"})   # replace the id with id of file you want to access
downloaded.GetContentFile('s.txt') 

# Creamos el Spark Context

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
type(sc)

# Lectura de datos en Spark

## Paralelizando una coleccion de python

In [ ]:
## creamos 1000 enteros en una lista
integersList = range(1,1001)
len(integersList)

1000

In [ ]:
## Paralelizamos la coleccion utilizando 8 particiones o slices
## Esta operacion es una transformacion de datos en un RDD
## Dado que Spark usa lazy evaluation, no corren jobs de Spark
## hasta el momento
integersListRDD = sc.parallelize(integersList, 8)
type(integersListRDD)

pyspark.rdd.PipelinedRDD

In [ ]:
## podemos ver tambien otra informacion interesante del RDD
## el numero de particiones
integersListRDD.getNumPartitions()

In [ ]:
## el conjunto de transformaciones que se aplica
integersListRDD.toDebugString()

In [ ]:
## para ver mas metodos disponibles del RDD
help(integersListRDD)

In [ ]:
integersListRDD.take(5)

In [ ]:
integersListRDD.count()

## Leyendo archivo con textFile

In [ ]:
rdd = spark.sparkContext.textFile('s.txt')

In [ ]:
rdd

In [ ]:
rdd.count()

In [ ]:
rdd.take(5)

## Leyendo datos con el sqlContext

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
dataframe = sqlContext.read.text('s.txt')

In [ ]:
dataframe

In [ ]:
rddCsv = dataframe.rdd

In [ ]:
rddCsv

In [ ]:
rddCsv.take(5)

Tambien se pueden leer archivos csv, json, parquet, jdbc, etc

# Acciones

## Count

Obtiene la cantidad de registros del RDD

In [ ]:
integersListRDD.count()

NameError: ignored

### Take

Obtiene los primeros n registros del RDD

In [ ]:
integersListRDD.take(5)

## Collect

Obtiene TODOS los registros del RDD. Esto es un potencial problema, ya que si los datos no son acotados va a sobrecargar el driver. Solo se debe ejecutar si de antemano conocemos que la cantidad de datos es acotada.

In [ ]:
integersListRDD.collect() # danger

## First

Obtiene el primer registro del RDD

In [ ]:
integersListRDD.first()

## TakeOrdered

Obtiene los primeros n registros en base a un orden indicado.

In [ ]:
integersListRDD.takeOrdered(5, key=lambda x: -x)

## TakeSample

Obtiene una muestra de n registros con o sin reemplazo.

In [ ]:
integersListRDD.takeSample(False, 5)

## Reduce

Obtiene un solo registro, combinando el resultado en base a una función dada.

Suma de todos los nros del RDD:

In [ ]:
integersListRDD.reduce(lambda a,b: a+b)

Número más grande del RDD:

In [ ]:
integersListRDD.reduce(lambda a,b: a if a > b else b)

## CountByKey

Cuenta ocurrencias de registros para cada clave.

En Spark para que un registro sea considerado con clave debe se una tupla de unicamente dos elementos. El primer elemento es la key y el segundo el valor. A su vez, la key y el valor pueden estar compuestos por tuplas.

Cuento cuántos nros múltiplo de 2 hay y cuántos no:

In [ ]:
integersListRDD.map(lambda x: (x % 2, 1)).countByKey()

# Transformaciones

### Map

Transforma cada registro en base a la función dada.

In [ ]:
integersListRDD.map(lambda x: x*2).take(5)

In [ ]:
integersListRDD.map(lambda x: (x % 2, x)).take(5)

## Filter

Filtra registros en base a la función dada.

In [ ]:
integersListRDD.filter(lambda x: x % 2 == 0).take(5)

In [ ]:
integersListRDD.filter(lambda x: x % 2 == 0).count()

## FlatMap

Similar a Map, pero cada registro puede generar 0, 1 o más registros.

Para cada registro original genero un nuevo registro con el nro, otro con el nro menos uno y otro con el registro más uno:

In [ ]:
integersFlat = integersListRDD.flatMap(lambda x: [(x), (x-1), (x+1)])
integersFlat.count()

3000

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Spagk2.ipynb')

## ReduceByKey

Combina los registros para una misma clave en base a una función de reduce.

La función de reduce debe ser **conmutativa** y **asociativa**.

Del RDD salida del flatMap cuento cuantos registros hay para cada nro:

In [ ]:
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).count()

In [ ]:
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).take(10)

In [ ]:
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a > b else b)

## GroupByKey

Agrupa los registros para cada clave. Es similar a reduceByKey pero con 
groupByKey se obtiene todos los registros para cada clave.

Solo se debe utilizar si es necesario la información de cada registro y la cantidad de registros por clave no es demasiado grande.

GroupByKey es una transformación costosa.

Si se desea realizar una agregación, usar reduceByKey. Usar groupByKey para hacer una agregación esta MAL.

Necesito saber cuales son los nros múltiplos de 2 y cuales no:

In [ ]:
integersListRDD.map(lambda x: (x % 2, x)).groupByKey().map(lambda x: (x[0], list(x[1]))).collect().take()

## Distinct

Elimina registros duplicados (todo el registro debe coincidir)

Del RDD trás aplicar flatMap, obtengo los registros únicos:

In [ ]:
integersFlat.distinct().count()


# Ejemplos de transformaciones y acciones con los textos de Shakespeare

## Leo de a líneas

In [ ]:
lines = spark.sparkContext.textFile('s.txt')

## Cantidad de líneas totales

In [ ]:
lines.count()

124614

## Primeras 10 líneas

In [ ]:
lines.take(10)

['1609',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '                     1',
 '  From fairest creatures we desire increase,']

## Obtengo las palabras de todas las líneas (flatMap)

In [ ]:
words = lines.flatMap(lambda x: x.split())

In [ ]:
words.take(10)

['1609',
 'THE',
 'SONNETS',
 'by',
 'William',
 'Shakespeare',
 '1',
 'From',
 'fairest',
 'creatures']

In [ ]:
words.count()

902892

## Contando palabras (reduceByKey)

In [ ]:
wordsCount = words.map(lambda x: (x.lower(),1))

In [ ]:
wordsCount.take(10)

[('1609', 1),
 ('the', 1),
 ('sonnets', 1),
 ('by', 1),
 ('william', 1),
 ('shakespeare', 1),
 ('1', 1),
 ('from', 1),
 ('fairest', 1),
 ('creatures', 1)]

In [ ]:
wordsCounted = wordsCount.reduceByKey(lambda x,y: x+y)

In [ ]:
wordsCounted.take(10)

[('shakespeare', 258),
 ('1', 13),
 ('fairest', 39),
 ('creatures', 27),
 ('we', 3210),
 ('increase,', 9),
 ('thereby', 21),
 ("beauty's", 30),
 ('rose', 44),
 ('never', 959)]

In [ ]:
#wordsCounted.takeOrdered(10, lambda x: -x[1])
wordsCounted.takeOrdered(10, lambda x: -x[1])

[('the', 27681),
 ('and', 26066),
 ('i', 19540),
 ('to', 18737),
 ('of', 18084),
 ('a', 14424),
 ('my', 12456),
 ('in', 10721),
 ('you', 10666),
 ('that', 10489)]

### Mal uso de groupByKey

In [ ]:
wordsCount.groupByKey().takeOrdered(10, lambda x: -1 * len(x[1]))

[('the', <pyspark.resultiterable.ResultIterable at 0x7f78102ae470>),
 ('and', <pyspark.resultiterable.ResultIterable at 0x7f78102ae978>),
 ('i', <pyspark.resultiterable.ResultIterable at 0x7f78102ae438>),
 ('to', <pyspark.resultiterable.ResultIterable at 0x7f78102ae9e8>),
 ('of', <pyspark.resultiterable.ResultIterable at 0x7f78102ae518>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7f78102aea20>),
 ('my', <pyspark.resultiterable.ResultIterable at 0x7f78102aea90>),
 ('in', <pyspark.resultiterable.ResultIterable at 0x7f78102ae588>),
 ('you', <pyspark.resultiterable.ResultIterable at 0x7f78102aeb00>),
 ('that', <pyspark.resultiterable.ResultIterable at 0x7f78102aeb70>)]

In [ ]:
wordsCount.groupByKey().map(lambda a: (a[0], list(a[1]))).take(5)

In [ ]:
wordsCount.groupByKey().filter(lambda x: len(x[1]) < 5).map(lambda a: (a[0], list(a[1]))).take(5)

[('riper', [1, 1, 1]),
 ('memory:', [1]),
 ("feed'st", [1, 1, 1]),
 ("light's", [1]),
 ('fuel,', [1])]

## Palabra más larga (reduce)

In [ ]:
words.reduce(lambda a, b: a if (len(a) > len(b)) else b)

## Palabras que empiezan con a (filter)

In [ ]:
wordsA = words.filter(lambda word: word.startswith('a'))

In [ ]:
wordsA.count()

63676

In [ ]:
wordsA.take(10)

['as', 'a', 'abundance', 'art', 'and', 'a', 'asked,', 'all', 'all', 'an']

## Palabras únicas que empiezan con a (distinct)

In [ ]:
wordsA.distinct().count()

2688

## Cantidad de palabras por frecuencia de repetición ordenados (sortByKey)

In [ ]:
wordsCounted.take(5)

[('shakespeare', 258),
 ('1', 13),
 ('fairest', 39),
 ('creatures', 27),
 ('we', 3210)]

In [ ]:
wordsFreq = wordsCounted.map(lambda x: (x[1],1))

In [ ]:
wordsFreq.take(10)

[(258, 1),
 (13, 1),
 (39, 1),
 (27, 1),
 (3210, 1),
 (9, 1),
 (21, 1),
 (30, 1),
 (44, 1),
 (959, 1)]

In [ ]:
wordsFreq.reduceByKey(lambda a,b: a+b).sortByKey().take(10) # sort es una operacion muy costosa!

In [ ]:
wordsFreq.reduceByKey(lambda a,b: a+b).takeOrdered(20, lambda x: -x[1])

[(1, 31072),
 (2, 8493),
 (3, 4342),
 (4, 2659),
 (5, 1822),
 (6, 1338),
 (7, 1053),
 (8, 779),
 (9, 700),
 (10, 549),
 (11, 514),
 (12, 391),
 (13, 371),
 (14, 305),
 (15, 285),
 (17, 255),
 (16, 248),
 (18, 200),
 (21, 179),
 (20, 169)]

# Transformaciones entre dos RDD

## Union

Obtiene la unión entre dos RDD.

In [ ]:
integersList2 = range(501,1501)
len(integersList2)

1000

In [ ]:
integersList2RDD = sc.parallelize(integersList2)

In [ ]:
integersList2RDD.count()

1000

In [ ]:
integersListRDD.count()

1000

In [ ]:
union = integersListRDD.union(integersList2RDD)

In [ ]:
union.take(5)

In [ ]:
union.count()

2000

## Intersection

Intersección entre dos RDD.

In [ ]:
intersection = integersListRDD.intersection(integersList2RDD)

In [ ]:
intersection.count()

500

In [ ]:
intersection.take(10) # no necesariamente estan ordenados

[510, 520, 530, 540, 550, 560, 570, 580, 590, 600]

In [ ]:
intersection.collect() 

## Subtract

Elimina del primer RDD los registros que aparezcan en el segundo.

In [ ]:
subtract = integersListRDD.subtract(integersList2RDD)

In [ ]:
subtract.count()

500

In [ ]:
subtract.collect()

## Joins

Con los joins se combinan dos RDD en base a las claves de los registros. Junta cada registro del primer RDD con cada registro del segundo RDD que tengan la misma clave. No agrupa, sino que es de a pares de registro.

In [ ]:
data_alumnos = [
  (1,'Damian'),
  (2,'Luis'),
  (3,'Martin'),
  (4,'Natalia'),
  (5,'Joaquin')
]

alumnos = sc.parallelize(data_alumnos)

In [ ]:
alumnos.collect()

[(1, 'Damian'), (2, 'Luis'), (3, 'Martin'), (4, 'Natalia'), (5, 'Joaquin')]

In [ ]:
data_materias_aprobadas = [
  (1, 'Algebra'),
  (2, 'Análisis Matemático'),
  (3, 'Análisis Matemático'),
  (200, 'Algebra'),
  (2, 'Física')
]

materias_aprobadas = sc.parallelize(data_materias_aprobadas)

In [ ]:
materias_aprobadas.collect()

[(1, 'Algebra'), (2, 'Análisis Matemático'), (200, 'Algebra'), (2, 'Física')]

### Inner Join (Join)

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) con todos los pares de elementos para cada key. (especificamente los que hay en comun por esa clave en ambos sets de datos)

In [ ]:
alumnos.join(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', 'Análisis Matemático'))]

### Left Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos del set de datos izquierdo estaran en el resultado del join.

In [ ]:
alumnos.leftOuterJoin(materias_aprobadas).collect()

[(4, ('Natalia', None)),
 (1, ('Damian', 'Algebra')),
 (5, ('Joaquin', None)),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', 'Análisis Matemático'))]

### Right Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos del set de datos derecho estaran en el resultado del join.

In [ ]:
alumnos.rightOuterJoin(materias_aprobadas).collect()

[(200, (None, 'Algebra')),
 (1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', 'Análisis Matemático'))]

In [ ]:
materias_aprobadas.leftOuterJoin(alumnos).collect()

[(200, ('Algebra', None)),
 (1, ('Algebra', 'Damian')),
 (2, ('Análisis Matemático', 'Luis')),
 (2, ('Física', 'Luis')),
 (3, ('Análisis Matemático', 'Martin'))]

### Outer/Full Join


Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos de ambos set de datos estaran aunque no haya match de keys.

In [ ]:
alumnos.fullOuterJoin(materias_aprobadas).collect()

[(4, ('Natalia', None)),
 (200, (None, 'Algebra')),
 (1, ('Damian', 'Algebra')),
 (5, ('Joaquin', None)),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', 'Análisis Matemático'))]

### Broadcast Join (map-side join)

#### Variable Broadcast

Una variable Broadcast nos permite mantener una variable solo lectura cacheada en cada una de las maquinas del cluster en vez de enviar esa informacion con cada una de las tareas que se envian al cluster.

Esto es particularmente util cuando cuando tareas a partir de multiples etapas (stages) necesitan la misma información o cuando cachear información de forma deserializada es importante.

Tener en cuenta que esto **es posible** cuando uno de los data sets o conjunto de datos **es lo suficientemente pequeño para ser broadcasteado a todos los nodos/workers del cluster**.

In [ ]:
# Vamos a suponer que tenemos un RDD de productos por sus IDs identificando ventas de los mismos
prodsList = [1,11,1,4,5,11,2,3,4,5,6,4,5,4,3,2,1,11,2,3,4,5,6,4,3,2,1,1]
prods = sc.parallelize(prodsList,3)

In [ ]:
# Un hash con los productos y sus nombres: este rdd es por lo general acotado y mucho mas chico que el rdd de ventas
productNames = {1:'papas',
                2:'cebollas',
                3:'tomates',
                4:'zanahorias',
                5:'batatas',
                6:'peras',
                7:'cilantro',
                8:'apio',
                9:'morrones',
                10:'manzanas',
                11:'naranjas'}

# Hacemos un broadcast de la variable
bproductNames = sc.broadcast(productNames)

In [ ]:
# Buscamos los productos que se vendieron más de 4 veces
popularProds = prods.map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .filter(lambda x:x[1]>=4)
popularProds.collect()

[(3, 4), (1, 5), (4, 6), (5, 4), (2, 4)]

El join se realiza de forma implicita usando un map y dentro del mismo accediendo a la informacion de la variable a la que se realizo el broadcast via .value

In [ ]:
popularProds = popularProds.map(
    lambda x:(bproductNames.value[x[0]],x[0],x[1]))
popularProds.collect()

[('tomates', 3, 4),
 ('papas', 1, 5),
 ('zanahorias', 4, 6),
 ('batatas', 5, 4),
 ('cebollas', 2, 4)]

#### Ventajas

Cuando un valor es "broadcasteado" al cluster, este es copiado a los nodos/workers **sólo una vez** (en vez de múltiples veces si la información fuera a enviarse en cada task). De esta forma se resuelve la consulta más rapidamente.

# Transformaciones sobre las particiones

In [ ]:
rdd = sc.parallelize(range(1,11))
rdd.getNumPartitions()

2

In [ ]:
sc.defaultParallelism

2

In [ ]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## Glom

Junta los registros de cada partición en una lista.

In [ ]:
rdd.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

## MapPartitions

Devuelve un nuevo RDD aplicando una función a cada partición del RDD.

In [ ]:
def f(iterator): yield __builtin__.sum(iterator)
rdd.mapPartitions(f).collect()

[15, 40]

## Repartition

Reshuffle los datos en el RDD de forma aleatoria para crear más o menos particiones y balancearlas. 

Hace un shuffle de todo los datos por la red.

In [ ]:
rdd = sc.parallelize(range(1,11), 4)
rdd.getNumPartitions()

4

In [ ]:
rdd.glom().collect()

[[1, 2], [3, 4, 5], [6, 7], [8, 9, 10]]

In [ ]:
rdd2 = rdd.repartition(2)

In [ ]:
rdd2.getNumPartitions()

2

In [ ]:
rdd2.glom().collect()

[[1, 2, 6, 7, 8, 9, 10], [3, 4, 5]]

Spark no hace shuffle de registros individuales sino de a bloques con un mínimo (no es un problema cuando se manejan grandes cantidades de datos)

## Coalesce

Decrementa la cantidad de particiones del RDD.

No hace shuffle por defecto, solo pasa datos de una partición a otra.

No quedan balanceadas.

In [ ]:
rddCoalesce = rdd.coalesce(2)

In [ ]:
rddCoalesce.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

## RepartitionAndSortWithinPartitions

Reparticiona un RDD de acuerdo a un particionador y ordena los registros en base a su clave.

Los registros deben tener clave.

Es más eficiente que hacer un repartition y luego un sort dentro de cada partición ya que realiza el sort en el mismo paso de shuffle.

In [ ]:
rdd.map(lambda x: (x, x)).collect()

[(1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10)]

In [ ]:
rdd.map(lambda x: (x, x)).glom().collect()

[[(1, 1), (2, 2)],
 [(3, 3), (4, 4), (5, 5)],
 [(6, 6), (7, 7)],
 [(8, 8), (9, 9), (10, 10)]]

### Ascending

In [ ]:
rdd.map(lambda x: (x, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 2), (4, 4), (6, 6), (8, 8), (10, 10)],
 [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]

In [ ]:
rdd.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(0, 3), (0, 6), (0, 9), (2, 2), (2, 5), (2, 8)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

In [ ]:
rdd.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2, ascending=False).glom().collect()

[[(2, 2), (2, 5), (2, 8), (0, 3), (0, 6), (0, 9)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

### PartitionFunc

In [ ]:
rdd.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 1),
  (4, 2),
  (6, 3),
  (8, 4),
  (10, 5),
  (12, 6),
  (14, 7),
  (16, 8),
  (18, 9),
  (20, 10)],
 []]

In [ ]:
rdd.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2, partitionFunc=lambda x: (x % 3)).glom().collect()

[[(2, 1),
  (6, 3),
  (8, 4),
  (12, 6),
  (14, 7),
  (18, 9),
  (20, 10),
  (24, 12),
  (26, 13),
  (30, 15),
  (32, 16),
  (36, 18),
  (38, 19),
  (42, 21),
  (44, 22),
  (48, 24),
  (50, 25),
  (54, 27),
  (56, 28),
  (60, 30),
  (62, 31),
  (66, 33),
  (68, 34),
  (72, 36),
  (74, 37),
  (78, 39),
  (80, 40),
  (84, 42),
  (86, 43),
  (90, 45),
  (92, 46),
  (96, 48),
  (98, 49),
  (102, 51),
  (104, 52),
  (108, 54),
  (110, 55),
  (114, 57),
  (116, 58),
  (120, 60),
  (122, 61),
  (126, 63),
  (128, 64),
  (132, 66),
  (134, 67),
  (138, 69),
  (140, 70),
  (144, 72),
  (146, 73),
  (150, 75),
  (152, 76),
  (156, 78),
  (158, 79),
  (162, 81),
  (164, 82),
  (168, 84),
  (170, 85),
  (174, 87),
  (176, 88),
  (180, 90),
  (182, 91),
  (186, 93),
  (188, 94),
  (192, 96),
  (194, 97),
  (198, 99),
  (200, 100),
  (204, 102),
  (206, 103),
  (210, 105),
  (212, 106),
  (216, 108),
  (218, 109),
  (222, 111),
  (224, 112),
  (228, 114),
  (230, 115),
  (234, 117),
  (236, 118),
  

# Persistiendo RDD

## Cache

Cachea un RDD intermedio que va a ser utilizado varias veces de modo de evitar tener que ejecutar todas las transformaciones cada vez.

In [ ]:
rdd = sc.parallelize(range(1,100000))

In [ ]:
rddCached = rdd.map(lambda x: x*10).cache()

In [ ]:
rddCached.count()

99999

In [ ]:
rddCached.take(10)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

## SaveAsTextFile

Guarda un RDD a disco en un archivo de texto.

In [ ]:
rdd.saveAsTextFile('numbers.txt')

In [ ]:
rddN = sc.textFile('numbers.txt')

In [ ]:
rddN.collect()

## SaveAsPickleFile

Guarda un RDD a disco en un archivo con los datos serializados.

In [ ]:
rdd.saveAsPickleFile('numbers2.file')

In [ ]:
rddN2 = sc.pickleFile('numbers2.file')

In [ ]:
rddN2.take(10)